In [2]:
### pip install and importing packages ###
!sudo apt-get install python3.7
!sudo apt-get update -y
!sudo update-alternatives --install /usr/bin/python3 python3
!sudo update-alternatives --config python3

! pip install recourse==1.0.0
! pip install lime==0.2.0.1

!pip install wandb==0.13.4
!pip install nvidia-ml-py3

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CurrentWork/CARLA - PPM


'sudo' is not recognized as an internal or external command,
operable program or batch file.
'sudo' is not recognized as an internal or external command,
operable program or batch file.
'sudo' is not recognized as an internal or external command,
operable program or batch file.
'sudo' is not recognized as an internal or external command,
operable program or batch file.


     ---------------------------------------- 1.9/1.9 MB 29.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21486 sha256=8aed5218d934197370750922b303b0d4db77bf45cfeb7de0306ea7d06a465b7e
  Stored in directory: c:\users\u0138175\appdata\local\pip\cache\wheels\29\93\c6\762e359f8cb6a5b69c72235d798804cae523bbe41c2aa8333d
Successfully built promise
  Attempting uninstall: wandb
    Found existing installation: wandb 0.15.10
    Uninstalling wandb-0.15.10:
      Successfully uninstalled wandb-0.15.10
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19172 sha256=fcbc49624af41c055ecaa10006880e12155f19c1ec83d6a4726ff490b49542c2
  Stored in directory: c:\users\u0138175\appdata\local\pip\cache\wheels\df\99\da\c34f202

In [3]:
# set logging
import logging
logging.getLogger().setLevel(logging.INFO)
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import sys
sys.path.append(os.getcwd())

import wandb
import yaml
import numpy as np
import torch
from torch.utils.data import DataLoader
from carla import log
from carla.iBCM.check_constraint_violation import ProcessConstraints
from carla.data.catalog.own_catalog import OwnCatalog
from carla.recourse_methods.autoencoder.models import VariationalAutoencoder, CheckpointSaver
from util.DatasetManager import DatasetManager
from util.settings import global_setting, training_setting
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
device = "cuda" if torch.cuda.is_available() else "cpu" # Use GPU if available
#torch.autograd.set_detect_anomaly(True) # to explicitly raise an error with a stack trace to easier debug which operation might have created the invalid values

Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [4]:
##################################################################
# PARAMETERS
seed = global_setting['seed']
epochs = training_setting['epochs_VAE']
n_splits = global_setting['n_splits']
max_evals = global_setting['max_evals']
train_ratio = global_setting['train_ratio']
clip = training_setting["clip"]
lambda_reg=1e-6
kl_weight = 0.3
dataset_names = ['production','sepsis_cases_1', 'sepsis_cases_2', 'sepsis_cases_4', 'bpic2012_accepted', 'bpic2012_declined', 'bpic2012_cancelled']
dataset_name = 'production'

with open("carla/hyperoptimization/config/VAE_sweep.yaml", 'r') as stream:
    sweep_config = yaml.safe_load(stream)

print('own dataset', dataset_name)
dataset = OwnCatalog(dataset_name)
dataset_manager = DatasetManager(dataset_name)
# we only use the training dataset for hyperoptimization
data_train = dataset.df_train
target_train = dataset.target_train
target = [0]
recon_loss_function = torch.nn.CrossEntropyLoss(ignore_index=0)
print('shape', data_train.shape)

"""
Here you can change the parameter that decides whether you take the joint constraints in the loss function of the VAE or not.
"""
# set to True if you want them in the loss.
joint_constraint_in_loss = True

if joint_constraint_in_loss == True:
    #search for traces where there are any constraint violations
    constraintminer = ProcessConstraints(dataset_name,dataset.vocab_size, target)
    #paths
    path = 'carla/hyperoptimization/params_dir_VAE'
    dir_path = path+'/hyper_VAEs/'+dataset_name
    project_name = 'counterfactuals_LSTMVAE'

elif joint_constraint_in_loss == False:
    constraintminer = None
    path = 'carla/hyperoptimization/params_dir_VAE_no_PC'
    dir_path = path+'/hyper_VAEs_no_PC/'+dataset_name
    project_name = 'counterfactuals_LSTMVAE_no_PC'

dataset_manager.ensure_path(path)
dataset_manager.ensure_path(dir_path)
os.environ['WANDB_DIR'] = dir_path
os.environ["WANDB_SILENT"] = "true"

#wandb project
dataset_group = 'VAE'
name = dataset_group + '_' + dataset_name

#######WANDB################
%cd /content/drive/MyDrive/CurrentWork/CARLA - PPM

sweep_config['name'] = name
sweep_id = wandb.sweep(sweep_config, project=project_name, entity="adversarial_robustness")

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config, project=project_name, group=name, save_code=False) as run:
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        ###################MODEL ARCHITECTURE#################################################
        train_data = torch.LongTensor(dataset.df_train.long()).to(device)
        train_data = DataLoader(train_data, batch_size= 128, shuffle=True, drop_last=True)
        model = VariationalAutoencoder(dataset_name, config['lstm_size'], dataset.vocab_size, dataset.max_prefix_length, epochs, constraintminer, joint_constraint_in_loss).to(device)
        model = model.to(device)
        # Epochs
        optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=lambda_reg)
        # checkpoint saver
        checkpoint_saver = CheckpointSaver(dirpath=dir_path, decreasing=True, top_n=1)
        # Train the VAE with the new prior
        ELBO = np.zeros((epochs, 1))
        log.info("Start training of Variational Autoencoder...")
        print(config)
        for epoch in range(epochs):
          print('epoch', epoch, 'of', epochs)
          beta = epoch * kl_weight / epochs
          beta2 = 1
          # Train for all the batches

          for data in train_data:
            model.train()
            data = data.to(device).float()
            padding_tensor = torch.zeros([dataset.vocab_size]).to(device)
            padding_tensor[0] =1
            mask = ~torch.all(torch.eq(data.long(), padding_tensor), dim=2).to(device)
            data = data * mask.unsqueeze(-1).float()  # Apply the mask to ignore padded sequences
            data = data.to(device).float()
            optimizer.zero_grad() # Update the parameters
            reconstruction, mu, log_var = model(data) # returns the reconstruction, mu and log_var
            reconstruction  = model.mask_out_tensor(reconstruction)
            class_indices = torch.argmax(data, dim=2)
            class_indices = class_indices.view(-1).to(device)
            reconstruction_constraint = reconstruction.clone().detach().cpu()
            reconstruction = reconstruction.view(-1, dataset.vocab_size).to(device)
            recon_loss = recon_loss_function(reconstruction, class_indices)
            kld_loss = model.kld(mu, log_var).to(device)
            if joint_constraint_in_loss==True:
              constraint_loss = model.constraint_violation.calculate_joint_constraint_loss(reconstruction_constraint)
              #constraint_loss = constraint_loss.to("cpu")
              loss = recon_loss + beta * kld_loss + beta2 * constraint_loss
            else:
              loss = recon_loss + beta * kld_loss
            loss = loss.to(device)
            loss.backward() # Compute the loss
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0) #gradient clipping to avoid gradients from exploding
            # Update the parameters
            optimizer.step()
          with torch.no_grad():
            model.eval()
            padding_tensor = padding_tensor.to(device)
            test = dataset.df_train.to(device).float()
            mask_test = ~torch.all(torch.eq(test, padding_tensor), dim=2).to(device)
            test = test * mask_test.unsqueeze(-1).float()  # Apply the mask to ignore padded sequences
            test = test.to(device)
            assert not torch.isnan(test).any(), "NaN values detected in training data!"
            reconstruction_test, mu_test, log_var_test = model(test) # returns the reconstruction, mu and log_var
            assert not torch.isnan(reconstruction_test).any(), "NaN values detected in reconstruction!"
            reconstruction_test  = model.mask_out_tensor(reconstruction_test)
            class_indices_test = torch.argmax(test, dim=2)
            class_indices_test = class_indices_test.view(-1).to(device)
            reconstruction_constraint_test = reconstruction_test.clone().detach().cpu()
            reconstruction_test = reconstruction_test.view(-1, dataset.vocab_size).to(device)
            recon_loss_test = recon_loss_function(reconstruction_test, class_indices_test).to(device)
            kld_loss_test = model.kld(mu_test, log_var_test).to(device)
            if joint_constraint_in_loss==True:
              constraint_loss_test = model.constraint_violation.calculate_joint_constraint_loss(reconstruction_constraint_test)
              #constraint_loss_test = constraint_loss_test.to("cpu")
              validation_loss = recon_loss_test + beta * kld_loss_test + beta2 * constraint_loss_test
              #print('recon loss', recon_loss_test, 'kld', kld_loss_test, 'constraints', constraint_loss_test)
            else:
              validation_loss = recon_loss_test + beta * kld_loss_test
          checkpoint_saver(model, epoch, validation_loss, config['learning_rate'], config["lstm_size"], 'adam')
          wandb.log({"validation_loss": validation_loss})
        # Get the lowest validation loss of the training epochs
wandb.agent(sweep_id, function=train, count=max_evals, project=project_name, entity="adversarial_robustness")

own dataset production
[INFO] Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8. [utils.py _init_num_threads]
[INFO] NumExpr defaulting to 8 threads. [utils.py _init_num_threads]
the minimum prefix length in the dataset is 1
the dataset has min prefix length of 1 and max prefix length of 21
shape torch.Size([176, 22, 28])
total constraints []
the label-specific constraints are: []
the joint constraints are: ['exactly(27-27)', 'last(27-27)']
[WinError 3] The system cannot find the path specified: '/content/drive/MyDrive/CurrentWork/CARLA - PPM'
g:\My Drive\CurrentWork\CARLA - PPM
Create sweep with ID: 86kdioad
Sweep URL: https://wandb.ai/adversarial_robustness/counterfactuals_LSTMVAE/sweeps/86kdioad


In [9]:
print('helllo')

helllo


In [5]:
print(38)

38


In [6]:
vocab_size = 38

In [7]:
print(round(19,0))

19


In [8]:
print(round(vocab_size/2,0))

19.0
